In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, time
import matplotlib.pyplot as plt
import plotly.graph_objects as go

References:
- https://www.kite.com/python/answers/how-to-select-rows-by-multiple-label-conditions-with-pandas-loc-in-python
- https://stackoverflow.com/questions/50375985/pandas-add-column-with-value-based-on-condition-based-on-other-columns
- https://stackoverflow.com/questions/19384532/get-statistics-for-each-group-such-as-count-mean-etc-using-pandas-groupby
- https://stackoverflow.com/questions/44111307/python-pandas-count-rows-based-on-column
- https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.astype.html
- https://datatofish.com/line-chart-python-matplotlib/
- https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sort_values.html
- https://github.com/rahulrohri/final_project_2020Sp
- https://studio.mapbox.com/
- https://plotly.com/python/scattermapbox/

In [2]:
crashes = None
persons = None

In [3]:
NYC_collision_crashes_file = "Motor_Vehicle_Collisions_-_Crashes.csv"
NYC_collision_persons_file = "Motor_Vehicle_Collisions_-_Person.csv"

In [4]:
def load_collision_data(crashes, persons, crashes_file, persons_file):
    crashes = pd.read_csv(crashes_file, low_memory=False)
    persons = pd.read_csv(persons_file, low_memory=False)
    
    crashes.loc[:, 'CRASH_YEAR'] = crashes['CRASH DATE'].astype(np.str_).apply(lambda x: int(x.split('/')[-1]))
    
    return crashes, persons

In [5]:
crashes, persons = load_collision_data(crashes, persons, NYC_collision_crashes_file, NYC_collision_persons_file)

In [6]:
crashes.columns

Index(['CRASH DATE', 'CRASH TIME', 'BOROUGH', 'ZIP CODE', 'LATITUDE',
       'LONGITUDE', 'LOCATION', 'ON STREET NAME', 'CROSS STREET NAME',
       'OFF STREET NAME', 'NUMBER OF PERSONS INJURED',
       'NUMBER OF PERSONS KILLED', 'NUMBER OF PEDESTRIANS INJURED',
       'NUMBER OF PEDESTRIANS KILLED', 'NUMBER OF CYCLIST INJURED',
       'NUMBER OF CYCLIST KILLED', 'NUMBER OF MOTORIST INJURED',
       'NUMBER OF MOTORIST KILLED', 'CONTRIBUTING FACTOR VEHICLE 1',
       'CONTRIBUTING FACTOR VEHICLE 2', 'CONTRIBUTING FACTOR VEHICLE 3',
       'CONTRIBUTING FACTOR VEHICLE 4', 'CONTRIBUTING FACTOR VEHICLE 5',
       'COLLISION_ID', 'VEHICLE TYPE CODE 1', 'VEHICLE TYPE CODE 2',
       'VEHICLE TYPE CODE 3', 'VEHICLE TYPE CODE 4', 'VEHICLE TYPE CODE 5',
       'CRASH_YEAR'],
      dtype='object')

In [7]:
persons.columns

Index(['UNIQUE_ID', 'COLLISION_ID', 'CRASH_DATE', 'CRASH_TIME', 'PERSON_ID',
       'PERSON_TYPE', 'PERSON_INJURY', 'VEHICLE_ID', 'PERSON_AGE', 'EJECTION',
       'EMOTIONAL_STATUS', 'BODILY_INJURY', 'POSITION_IN_VEHICLE',
       'SAFETY_EQUIPMENT', 'PED_LOCATION', 'PED_ACTION', 'COMPLAINT',
       'PED_ROLE', 'CONTRIBUTING_FACTOR_1', 'CONTRIBUTING_FACTOR_2',
       'PERSON_SEX'],
      dtype='object')

## Hypothesis:
#### Of all collisions occurring late in the night (between 12 am - 5 am), the majority are caused due to overspeeding.

In [8]:
night_crash_data = None

In [9]:
def get_night_crashes(crashes):
    
    time_data = crashes['CRASH TIME']
    crashes['CRASH TIME'] = crashes['CRASH TIME'].apply(lambda x: datetime.strptime(x, "%H:%M").time())

    night_crash_data = crashes[(crashes['CRASH TIME'] < time(5, 0, 0))]

    return night_crash_data

In [10]:
def check_for_unsafe_speed(night_crash_data):
    night_crash_data = night_crash_data.assign(hasUnsafeSpeed=False)
    night_crash_data.loc[((night_crash_data['CONTRIBUTING FACTOR VEHICLE 1']=='Unsafe Speed') | 
                          (night_crash_data['CONTRIBUTING FACTOR VEHICLE 2']=='Unsafe Speed') | 
                          (night_crash_data['CONTRIBUTING FACTOR VEHICLE 3']=='Unsafe Speed') | 
                          (night_crash_data['CONTRIBUTING FACTOR VEHICLE 4']=='Unsafe Speed') | 
                          (night_crash_data['CONTRIBUTING FACTOR VEHICLE 5']=='Unsafe Speed')), 
                         'hasUnsafeSpeed'] = True
    
    return night_crash_data

In [11]:
def calculate_percentage_of_speedy_collisions(night_crash_data):
    unsafe_speed_metrics = night_crash_data['hasUnsafeSpeed'].value_counts().to_frame()
    percentage_unsafe_speed_collisions = unsafe_speed_metrics.iloc[1]*100/night_crash_data.shape[0]
    
    return percentage_unsafe_speed_collisions

In [12]:
def calculate_invalid_collision_percentage(night_crash_data):
    
    unwanted_contributing_factors = ['1','80','Unspecified']
    
    night_crash_data['isUnspecified'] = np.where((((
        night_crash_data['CONTRIBUTING FACTOR VEHICLE 1'].isin(unwanted_contributing_factors)) | 
        (night_crash_data['CONTRIBUTING FACTOR VEHICLE 1'].isnull())) & 
        ((night_crash_data['CONTRIBUTING FACTOR VEHICLE 2'].isin(unwanted_contributing_factors)) | 
         (night_crash_data['CONTRIBUTING FACTOR VEHICLE 2'].isnull())) & 
        ((night_crash_data['CONTRIBUTING FACTOR VEHICLE 3'].isin(unwanted_contributing_factors))  | 
         (night_crash_data['CONTRIBUTING FACTOR VEHICLE 3'].isnull())) & 
        ((night_crash_data['CONTRIBUTING FACTOR VEHICLE 4'].isin(unwanted_contributing_factors))  | 
         (night_crash_data['CONTRIBUTING FACTOR VEHICLE 4'].isnull())) & 
        ((night_crash_data['CONTRIBUTING FACTOR VEHICLE 5'].isin(unwanted_contributing_factors))  | 
         (night_crash_data['CONTRIBUTING FACTOR VEHICLE 5'].isnull()))), True, False)
    
    invalid_night_crash_data_metrics = night_crash_data['isUnspecified'].value_counts().to_frame()
    
    percentage_invalid_collision_data = invalid_night_crash_data_metrics.iloc[1]*100/night_crash_data.shape[0]
    
    return percentage_invalid_collision_data

In [13]:
night_crash_data = get_night_crashes(crashes)

In [14]:
night_crash_unsafe_speed_data = check_for_unsafe_speed(night_crash_data)

In [15]:
percentage_unsafe_speed_collisions = calculate_percentage_of_speedy_collisions(night_crash_unsafe_speed_data)

In [16]:
percentage_unsafe_speed_collisions

hasUnsafeSpeed    2.687928
Name: True, dtype: float64

In [17]:
percentage_invalid_collision_data = calculate_invalid_collision_percentage(night_crash_data)

C:\Users\agarw\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [18]:
percentage_invalid_collision_data

isUnspecified    34.79346
Name: True, dtype: float64

## Hypothesis:

#### Of all crashes, a majority number is caused by persons between the age of 16-25.

In [20]:
def get_merged_crashes_persons(crashes, persons):
    crashes_persons = pd.merge(crashes, persons, left_on='COLLISION_ID', right_on='COLLISION_ID', how='inner')
    crashes_persons.loc[:, 'CRASH_YEAR'] = crashes_persons['CRASH_DATE'].astype(np.str_).apply(lambda x: x.split('/')[-1])
    del crashes_persons['CRASH_DATE']
    del crashes_persons['CRASH_TIME']
    del crashes_persons['UNIQUE_ID']
    return crashes_persons

In [21]:
crashes_persons = get_merged_crashes_persons(crashes, persons)

In [22]:
crashes_persons.columns

Index(['CRASH DATE', 'CRASH TIME', 'BOROUGH', 'ZIP CODE', 'LATITUDE',
       'LONGITUDE', 'LOCATION', 'ON STREET NAME', 'CROSS STREET NAME',
       'OFF STREET NAME', 'NUMBER OF PERSONS INJURED',
       'NUMBER OF PERSONS KILLED', 'NUMBER OF PEDESTRIANS INJURED',
       'NUMBER OF PEDESTRIANS KILLED', 'NUMBER OF CYCLIST INJURED',
       'NUMBER OF CYCLIST KILLED', 'NUMBER OF MOTORIST INJURED',
       'NUMBER OF MOTORIST KILLED', 'CONTRIBUTING FACTOR VEHICLE 1',
       'CONTRIBUTING FACTOR VEHICLE 2', 'CONTRIBUTING FACTOR VEHICLE 3',
       'CONTRIBUTING FACTOR VEHICLE 4', 'CONTRIBUTING FACTOR VEHICLE 5',
       'COLLISION_ID', 'VEHICLE TYPE CODE 1', 'VEHICLE TYPE CODE 2',
       'VEHICLE TYPE CODE 3', 'VEHICLE TYPE CODE 4', 'VEHICLE TYPE CODE 5',
       'CRASH_YEAR', 'PERSON_ID', 'PERSON_TYPE', 'PERSON_INJURY', 'VEHICLE_ID',
       'PERSON_AGE', 'EJECTION', 'EMOTIONAL_STATUS', 'BODILY_INJURY',
       'POSITION_IN_VEHICLE', 'SAFETY_EQUIPMENT', 'PED_LOCATION', 'PED_ACTION',
       'COMPL

In [23]:
crashes_persons['PERSON_TYPE'].unique()

array(['Occupant', 'Pedestrian', 'Bicyclist'], dtype=object)

In [25]:
# check if there are entries where person is of 'Bicyclist' type and vehicle id is None
crashes_persons[(crashes_persons['VEHICLE_ID'].isna()) & 
                (crashes_persons['PERSON_TYPE'] == 'Bicyclist')][['COLLISION_ID', 'PERSON_TYPE', 'POSITION_IN_VEHICLE']]

,COLLISION_ID,PERSON_TYPE,POSITION_IN_VEHICLE


In [26]:
# dropping all rows where there is no vehicle ID present
crashes_persons.drop(crashes_persons.loc[crashes_persons['VEHICLE_ID'].isna()].index, inplace=True)

In [32]:
def get_crashes_persons_age_grouping_data(crashes_persons, columns):
    crashes_persons_age_grouping = crashes_persons[crashes_persons['POSITION_IN_VEHICLE'] == 'Driver'][columns]
    crashes_persons_age_grouping.loc[:, 'age16-25'] = np.where((crashes_persons_age_grouping['PERSON_AGE'] > 15) & 
                                                               (crashes_persons_age_grouping['PERSON_AGE'] < 26), True, False)
    return crashes_persons_age_grouping

In [33]:
columns = ['COLLISION_ID', 'VEHICLE_ID', 'PERSON_TYPE', 'POSITION_IN_VEHICLE', 'PERSON_AGE']
crashes_persons_age_grouping = get_crashes_persons_age_grouping_data(crashes_persons, columns)

In [34]:
crashes_persons_age_grouping.head()

,COLLISION_ID,VEHICLE_ID,PERSON_TYPE,POSITION_IN_VEHICLE,PERSON_AGE,age16-25
1,3916262,18655232.0,Occupant,Driver,57.0,False
5,3928600,18762804.0,Occupant,Driver,38.0,False
6,3928600,18762803.0,Occupant,Driver,39.0,False
8,3921967,18643741.0,Occupant,Driver,43.0,False
9,3921967,18643742.0,Occupant,Driver,83.0,False


In [37]:
crashes_persons_age_grouping[crashes_persons_age_grouping['PERSON_AGE'] < 16]['PERSON_AGE'].unique()

array([   0.,   15.,   10.,   13.,   12.,    2.,    8.,    1., -947.,
       -518.,    9., -936.,   11., -597.,   14., -968., -970., -594.,
       -965., -593.,    3.,    7., -938., -534., -296.,    6., -177.,
       -978.,    5., -595., -798., -714.,    4., -953., -217., -944.,
       -417., -517., -596., -592., -964., -797., -617., -969., -935.,
       -962., -533., -977., -959., -955., -963., -974., -973., -960.,
       -966., -536., -178., -971., -967., -972., -948., -899., -942.,
       -951., -975., -941., -945., -184., -618., -931., -957., -946.,
       -717., -934., -954., -626., -949., -976., -117., -958., -933.,
       -961., -294., -722., -194., -956., -418., -317., -716., -537.,
       -179., -939., -922., -999., -979., -943., -293., -181., -180.,
       -950., -198., -199., -718., -715., -174., -982., -927., -930.,
       -255., -952., -929., -295.])

In [41]:
crashes_persons_age_grouping.loc[:, 'ageBelow16'] = np.where(crashes_persons_age_grouping['PERSON_AGE'] < 16, True, False)
crashes_persons_age_grouping['ageBelow16'].value_counts()

False    1465016
True        8528
Name: ageBelow16, dtype: int64

In [42]:
# Since there are only 8528 such rows where the age is below 16, we will be dropping those from the analysis
crashes_persons_age_grouping.drop(
    crashes_persons_age_grouping.loc[crashes_persons_age_grouping['PERSON_AGE'] < 16].index, 
    inplace=True)

In [43]:
crashes_persons_age_grouping['age16-25'].value_counts()

False    1257987
True      207029
Name: age16-25, dtype: int64

In [47]:
def calculate_percentage_drivers_between_ages_16_25(crashes_persons_age_grouping):
    df = crashes_persons_age_grouping['age16-25'].value_counts().to_frame()
    pct = df.iloc[1]*100/crashes_persons_age_grouping.shape[0]
    return pct

In [48]:
calculate_percentage_drivers_between_ages_16_25(crashes_persons_age_grouping)

age16-25    14.131518
Name: True, dtype: float64

## Hypothesis

#### The number of collisions increased with an increase in population

Source - https://worldpopulationreview.com/us-cities/new-york-city-ny-population

In [ ]:
def get_NYC_population_data():
    #Source - https://worldpopulationreview.com/us-cities/new-york-city-ny-population
    
    NYC_Population_data = {'Year':  [2012,2013,2014,2015,2016,2017,2018,2019,2020],
        'Population': [8348030,8398740,8437390,8468180,8475980,8438270, 8398750, 8361040, 8323340],
        }

    NYC_Population = pd.DataFrame (NYC_Population_data, columns = ['Year','Population'])
    
    NYC_area = 300.4 #(in sq miles) Source - https://worldpopulationreview.com/us-cities/new-york-city-ny-population
    
    NYC_Population['Population_Density'] = NYC_Population['Population'].apply(lambda x: x/NYC_area)
    
    return NYC_Population

In [ ]:
def get_total_crashes_per_year(crashes):
    crashes_data = crashes.copy()
    crashes_total = crashes_data.groupby(['CRASH_YEAR'], sort=False).size().reset_index(name='Total_Crashes')
    crashes_total['CRASH_YEAR'] = crashes_total['CRASH_YEAR'].astype('int64')
    
    crashes_total = crashes_total.sort_values(by=['CRASH_YEAR'])
    
    return crashes_total

In [ ]:
def calculate_crashes_per_capita(crashes_total, NYC_Population):
    crashes_population = pd.merge(crashes_total, NYC_Population, left_on='CRASH_YEAR', right_on='Year', how='inner')
    
    crashes_population.loc[:, 'Crashes_per_capita'] = crashes_population['Total_Crashes']/crashes_population['Population']
    
    return crashes_population

In [ ]:
def plot_crashes_per_capita_vs_year(crashes_population):
    plt.plot(crashes_population['Year'], crashes_population['Crashes_per_capita'], color='red', marker='o')
    plt.title('Crashes_per_Capita Vs Year for NYC')
    plt.xlabel('Year')
    plt.ylabel('Crashes_per_capita')
    plt.show()

In [ ]:
def plot_crashes_per_capita_vs_population_density(crashes_population):
    plt.plot(crashes_population['Crashes_per_capita'], crashes_population['Population_Density'], color='red', marker='o')
    plt.title('Crashes_per_Capita Vs Population for NYC')
    plt.xlabel('Population Density')
    plt.ylabel('Crashes_per_capita')
    plt.show()

In [ ]:
NYC_Population = get_NYC_population_data()

In [ ]:
crashes_total = get_total_crashes_per_year(crashes)

In [ ]:
crashes_population = calculate_crashes_per_capita(crashes_total, NYC_Population)

In [ ]:
plot_crashes_per_capita_vs_year(crashes_population_sorted)

In [ ]:
plot_crashes_per_capita_vs_population_density(crashes_population_sorted)

In [ ]:
crashes_population_subset = crashes_population.drop([0,8],0)

In [ ]:
plot_crashes_per_capita_vs_year(crashes_population_subset)

In [ ]:
plot_crashes_per_capita_vs_population_density(crashes_population_subset)

## Hypothesis

#### Crash locations are not random. The collisions are bound to specific areas due to a badly planned network of roads/traffic signs.

In [49]:
mapbox_access_token = 'pk.eyJ1IjoiYWdhcndhbGFkYXJzaCIsImEiOiJja2h5ZGYyd3UwZTN3MnFwYzM1YW9qNnFvIn0.SasVV15822weUxlZ3G0P8Q'

In [64]:
def prepare_crashes_data_for_maps(crashes, column_name, separator='/'):
    crashes.loc[:, 'CRASH_MONTH'] = crashes[column_name].astype(np.str_).apply(lambda x: int(x.split(separator)[0]))
    return crashes

In [65]:
def plot_crash_locations(map_box_access_token, crashes, year, month, borough=''):
    
    if not 0 < month < 13:
        raise Exception('Please provide a valid month number (between 1 and 12)')
    if year < 2013:
        raise Exception('Year values above 2013 only')
        
    if not borough:
        df = crashes[(crashes['CRASH_YEAR'] == year) & 
                          (crashes['CRASH_MONTH'] == month)]
    else:
        df = crashes[(crashes['BOROUGH'] == borough.upper()) & (crashes['CRASH_YEAR'] == year) & 
                          (crashes['CRASH_MONTH'] == month)]
    
    lat = df['LATITUDE']
    lon = df['LONGITUDE']
    df_text = df['CONTRIBUTING FACTOR VEHICLE 1']
    
    fig = go.Figure(go.Scattermapbox(
            lat=lat.tolist(),
            lon=lon.tolist(),
            mode='markers',
            marker=go.scattermapbox.Marker(
                size=5
            ),
            text=df_text
    ))

    fig.update_layout(
        hovermode='closest',
        width=960,
        height=600,
        mapbox=dict(
            accesstoken=mapbox_access_token,
            bearing=0,
            center=go.layout.mapbox.Center(
                lat=40.7,
                lon=-74
            ),
            pitch=0,
            zoom=8
        )
    )
    return fig

In [62]:
crashes = prepare_crashes_data_for_maps(crashes, 'CRASH_DATE')

In [68]:
nyc_map_fig = plot_crash_locations(mapbox_access_token, crashes, 2014, 5)
nyc_map_fig.show()